Import necessary modules

In [1]:
import time
import network

import machine
import dht

from umqtt.simple import MQTTClient

configure the dht22 sensor being connected to Pin 4 (D2).

In [2]:
d=dht.DHT22(machine.Pin(4))

Prepare the onboard led at Pin 2 to be used as a status indicator. It is operated in PWM mode. Set frequency to 1000 Hz and the duty cycle to 0 which is turning the led on.

In [4]:
led=machine.PWM(machine.Pin(2), freq=1000)
led.duty(0)

Prepare a static network interface to join to an existing network

In [5]:
sta_if = network.WLAN(network.STA_IF)

Connect to the existing and open network called `pyfi`.

In [9]:
sta_if.connect("pyfi")

Wait for being successfully connected in a while loop. Indicated connection attempts with a flashing led.

In [5]:
while not sta_if.isconnected():
    for index in range(3):
        led.duty(0)
        time.sleep_ms(10)
        led.duty(1024)
        time.sleep_ms(10)
    time.sleep_ms(500)

Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
NameError: name 'sta_if' is not defined


Print out the network configuration in success.

In [11]:
print(sta_if.ifconfig())

('192.168.2.248', '255.255.255.0', '192.168.2.1', '192.168.2.1')


In order to transfer our measured temperature and humidity values we use the mosquitto protocol available in the umqtt library. We create a client named `utemphum` connecting to a mosquitto request broker expected to be available at `192.168.2.1` which is part of the wifi network we connected to.

In [12]:
client=MQTTClient("utemphum", "192.168.2.1") # pcm

Connect to the request broker

In [ ]:
client.connect()

In an endless loop trigger a new measurement and send the new data labeled with the topic `temphum` to the broker. After completion indicate success with a short flash of the led and sleep for a second before repeating the process.

In [ ]:
while True:
    d.measure()
    client.publish("temphum", "{0:g}:{1:g}".format(d.temperature(), d.humidity()))
    led.duty(0)
    time.sleep_ms(10)
    led.duty(1024)
    time.sleep(1)